# Neurona logística (PyTorch) 

## Objetivo
Entrenar **una neurona logística**  para que aprenda la tabla lógica **OR**.

Dado un par de entradas binarias $( (x_1, x_2) )$, el modelo predice la **probabilidad** de que $( y = 1 )$ (verdadero).  
Matemáticamente, la neurona calcula:

$$
\begin{align*}
z &= w_1 x_1 + w_2 x_2 + b \\
\hat{y} &= \sigma(z) = \frac{1}{1 + e^{-z}}
\end{align*}
$$

donde:

Los parámetros del modelo son:  
$w_1, w_2$ — **pesos** o parámetros aprendidos,  
$b$ — **sesgo (bias)** que ajusta la frontera de decisión,  
$\sigma(z)$ — **función sigmoide** que mapea cualquier número real al rango $(0,1)$.

In [ ]:
import torch
import torch.nn as nn


# Elegir el dispositivo de cómputo

# torch.device() permite definir si el modelo y los tensores se ejecutan en:
#  - "cuda" → GPU (si está disponible)
#  - "cpu"  → procesador normal
# torch.cuda.is_available() devuelve True si hay GPU compatible con CUDA.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")


# 1️. Definición de la capa lineal (modelo lineal)

# nn.Linear(in_features, out_features)
# Crea una transformación lineal:  y = X·W^T + b
#  - in_features = número de variables de entrada (x1, x2 → 2)
#  - out_features = número de salidas (una sola neurona → 1)
# Internamente guarda dos parámetros entrenables:
#  - weight → matriz de pesos (w1, w2)
#  - bias   → término independiente b
capa_lineal = nn.Linear(2, 1)


# 2️. Función de activación (sigmoide)

# nn.Sigmoid() aplica la función logística σ(z) = 1 / (1 + e^(-z))
# Convierte el resultado lineal (z) en una probabilidad entre 0 y 1.
# No recibe argumentos.
activacion = nn.Sigmoid()


# 3️. Construcción del modelo secuencial

# nn.Sequential() agrupa varias capas en orden:
#   - Primero aplica la capa lineal (z = WX + b)
#   - Luego la sigmoide (σ(z))
# to(device) mueve el modelo al CPU o GPU definido antes.
modelo = nn.Sequential(
    capa_lineal,
    activacion
).to(device)


# 4️. Datos de entrenamiento (Tabla OR)

# torch.tensor(data, device, dtype)
#   - data: lista o arreglo de valores numéricos
#   - device: en qué dispositivo se guardan (CPU o GPU)
#   - dtype: tipo de dato (float32 → número decimal)
#
# Entradas X: todas las combinaciones de (x1, x2)
# Salidas Y: resultados de x1 OR x2
X = torch.tensor([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], device=device, dtype=torch.float32)

Y = torch.tensor([
    [0],
    [1],
    [1],
    [1]
], device=device, dtype=torch.float32)


# 5️. Función de pérdida y optimizador

# nn.BCELoss() → Binary Cross Entropy Loss
#   - Calcula la diferencia entre las predicciones y los valores reales.
#   - Mide qué tan buena es la predicción de una probabilidad binaria.
#   - Fórmula: L = -[y·log(ŷ) + (1 - y)·log(1 - ŷ)]
fn_perdida = nn.BCELoss()

# torch.optim.SGD(params, lr)
#   - Implementa el método del Gradiente Descendente Estocástico.
#   - params: parámetros del modelo (pesos y sesgos)
#   - lr: tasa de aprendizaje (learning rate)
# Actualiza los parámetros del modelo en cada iteración:
#   θ := θ - η * ∇θ L
optimizador = torch.optim.SGD(modelo.parameters(), lr=0.1)


# 6️. Ciclo de entrenamiento

# Repite el proceso 10,000 veces para que el modelo aprenda (epochs)
for epoch in range(10000):

    # --- Forward pass ---
    # Calcula las salidas del modelo para las entradas X
    # Internamente: ŷ = σ(WX + b)
    y_pred = modelo(X)

    # --- Calcular pérdida ---
    # Compara las predicciones (y_pred) con los valores reales (Y)
    perdida = fn_perdida(y_pred, Y)
    
    # --- Limpiar gradientes ---
    # Antes de calcular nuevos gradientes, se limpian los anteriores
    optimizador.zero_grad()

    # --- Backpropagation ---
    # Calcula automáticamente las derivadas parciales (gradientes)
    # de la pérdida con respecto a cada parámetro del modelo
    perdida.backward()

    # --- Actualización de parámetros ---
    # Modifica los pesos (w1, w2) y el sesgo (b)
    # según la dirección del gradiente y la tasa de aprendizaje
    optimizador.step()
    
    # --- Monitoreo del entrenamiento ---
    # Cada 1000 iteraciones imprime la pérdida para ver la convergencia
    if epoch % 1000 == 0:
        print(f"Iteración {epoch} - Pérdida: {perdida.item():.4f}")


# 7️. Predicciones finales

# Después del entrenamiento, volvemos a pasar los datos por el modelo
# para ver qué probabilidades produce.
print("\n🔹 Predicciones después de entrenar:")
print(modelo(X))
# Deberían aproximarse a:
# [ [0.0], [0.99], [0.99], [1.0] ]


# 8️. Parámetros finales (pesos y sesgo)

# modelo.named_parameters() devuelve un iterable con:
#  - el nombre del parámetro ("weight" o "bias")
#  - el valor entrenado (tensor con los pesos aprendidos)
print("\n🔹 Parámetros aprendidos:")
for nombre, param in modelo.named_parameters():
    print(f"{nombre}: {param.data}")


Usando dispositivo: cpu
Iteración 0 - Pérdida: 0.7678
Iteración 1000 - Pérdida: 0.0898
Iteración 2000 - Pérdida: 0.0471
Iteración 3000 - Pérdida: 0.0315
Iteración 4000 - Pérdida: 0.0236
Iteración 5000 - Pérdida: 0.0188
Iteración 6000 - Pérdida: 0.0156
Iteración 7000 - Pérdida: 0.0134
Iteración 8000 - Pérdida: 0.0117
Iteración 9000 - Pérdida: 0.0104

🔹 Predicciones después de entrenar:
tensor([[0.0205],
        [0.9918],
        [0.9918],
        [1.0000]], grad_fn=<SigmoidBackward0>)

🔹 Parámetros aprendidos:
0.weight: tensor([[8.6612, 8.6605]])
0.bias: tensor([-3.8653])


## Resumen de las funciones usadas

- **`torch.device(type)`**  
  Define si se usará **CPU** o **GPU** para ejecutar el modelo y los tensores.  
  Argumento: `"cpu"` o `"cuda"`.

---

- **`torch.cuda.is_available()`**  
  Devuelve `True` si existe una **GPU compatible con CUDA** disponible en el sistema.  
  *(Sin argumentos).*

---

- **`nn.Linear(in_features, out_features)`**  
  Crea una capa lineal que aplica la transformación:  
  $$
  y = W X + b
  $$
  donde \( W \) son los **pesos** y \( b \) el **sesgo**.  
  Argumentos:  
  - `in_features`: número de entradas (columnas de \( X \)).  
  - `out_features`: número de salidas (neuronas de salida).

---

- **`nn.Sigmoid()`**  
  Aplica la función logística:  
  $$
  \sigma(z) = \frac{1}{1 + e^{-z}}
  $$
  Convierte un valor real \( z \) en una probabilidad entre 0 y 1.  
  *(Sin argumentos).*

---

- **`nn.Sequential(...)`**  
  Combina varias capas en orden, de modo que la salida de una es la entrada de la siguiente.  
  Argumento: lista de capas (por ejemplo `[nn.Linear(), nn.Sigmoid()]`).

---

- **`torch.tensor(data, device, dtype)`**  
  Crea un **tensor** (estructura de datos multidimensional).  
  Argumentos:  
  - `data`: lista o arreglo NumPy.  
  - `device`: `"cpu"` o `"cuda"`.  
  - `dtype`: tipo de dato, por ejemplo `torch.float32`.

---

- **`nn.BCELoss()`**  
  Calcula la **pérdida de entropía cruzada binaria**, que mide la diferencia entre las probabilidades predichas y las reales:  
  $$
  L = -[y \log(\hat{y}) + (1-y)\log(1-\hat{y})]
  $$
  *(Sin argumentos).*

---

- **`torch.optim.SGD(params, lr)`**  
  Implementa el **Gradiente Descendente Estocástico (SGD)**, actualizando los parámetros según:  
  $$
  \theta := \theta - \eta \nabla_\theta L
  $$
  Argumentos:  
  - `params`: parámetros del modelo (pesos y sesgos).  
  - `lr`: tasa de aprendizaje (\( \eta \)).

---

- **`optimizer.zero_grad()`**  
  Limpia los **gradientes acumulados** antes de calcular los nuevos.  
  *(Sin argumentos).*

---

- **`loss.backward()`**  
  Calcula los **gradientes** de la pérdida con respecto a cada parámetro del modelo usando *backpropagation*.  
  *(Sin argumentos).*

---

- **`optimizer.step()`**  
  Actualiza los parámetros (\( W \), \( b \)) usando los gradientes calculados.  
  *(Sin argumentos).*

---

- **`model(X)`**  
  Aplica el **modelo completo** (capa lineal + sigmoide) a las entradas \( X \) y devuelve las predicciones \( \hat{y} \).  
  Argumento: `X` — tensor con los datos de entrada.
